### SLR Data Preprocessing
Creates a csv file that has all the census tracts that will be impacted by a given Sea Level Rise scenario.<br>
create_slr_csv() has a print statement that updates progress whenever a file is added to the cvs. This can be commented out if wanted.
#### Required variables can be set before invoking with %run:
<strong>slr_rise</strong><br>
How many feet of sea level rise (from 1 to 10ft)<br>
<strong>SLRs</strong><br>
List of all file extensions for SLR files to download - (Dynamic JavaScript so didn't scrape)</strong><br>
<strong>fips_dict</strong><br>
Dictionary of State-FIPS codes for all states in coastal contiguous US<br>

In [26]:
#If running file seperate from Analysis.ipynb, might need to pip install geopandas
! pip install geopandas

In [16]:
#imports
import pandas as pd
import geopandas as gpd
import fiona

In [17]:
#Setting variables to default values if not set before using %run to run file

try:
    slr_feet
except:
    slr_feet = '1'

try:
    SLRs
except:  
    SLRs = ['AL/AL','CA/CA_EKA','CA/CA_LOX','CA/CA_MTR','CA/CA_SGX','CA/CA_ChannelIslands','CA/CA_Catalina',
        'NewEngland/CT','DE/DE','DC/DC','FL/FL_Pan_West','FL/FL_Pan_East','FL/FL_West_3','FL/FL_West_2',
        'FL/FL_West_1','FL/FL_SW','FL/FL_Keys','FL/FL_SE','FL/FL_East_1','FL/FL_East_2','FL/FL_NE',
        'GA/GA_South','GA/GA_North','LA/LA_LP','LA/LA_Delta','LA/LA_Central','LA/LA_CentralEast',
        'LA/LA_CentralNorth','LA/LA_West','NewEngland/ME_East','NewEngland/ME_West','MD/MD_Southeast',
        'MD/MD_East','MD/MD_North','MD/MD_West','MD/MD_Southwest','NewEngland/MA','MS/MS','NewEngland/NH',
        'NJ/NJ_Northern','NJ/NJ_Middle','NJ/NJ_Southern','NY/NY_Hudson','NY/NY_Metro','NY/NY_Suffolk',
        'NC/NC_Northern','NC/NC_Middle1','NC/NC_Middle2','NC/NC_Southern1','NC/NC_Southern2','OR/OR_MFR',
        'OR/OR_PQR','PA/PA','NewEngland/RI','SC/SC_North','SC/SC_Central','SC/SC_South','TX/TX_North1',
        'TX/TX_North2','TX/TX_Central','TX/TX_South1','TX/TX_South2','VA/VA_EasternShore','VA/VA_Northern',
        'VA/VA_Middle','VA/VA_Southern','WA/WA_PQR','WA/WA_SEW','WA/WA_PugetNW','WA/WA_PugetSW']

try:
    fips_dict
except: 
    fips_dict = {'AL':'01','CA':'06','CT':'09','DE':'10','DC':'11','FL':'12','GA':'13','LA':'22','ME':'23',
             'MD':'24','MA':'25','MS':'28','NH':'33','NJ':'34','NY':'36','NC':'37','OR':'41','PA':'42','RI':'44',
             'SC':'45','TX':'48','VA':'51','WA':'53'}

In [18]:
#Checking if SLR.csv file already exists, and creating it if it doesn't 

In [19]:
%%bash
if [ ! -e "data/SLR.csv" ]; then
    echo 'SLRs,STATEFP,COUNTYFP,TRACTCE,census_tract,SLR_ft' >> data/SLR.csv
fi

In [20]:
def find_slr_layer(slr, folder, slr_feet):
    '''Input is an slr file stem for folder of .gdbtable files, and level of slr_feet rise
    output is the slr file name corresponding to the slr_feet parameter layer'''
    file_list = !ls {folder}
    for file in file_list:
        if fiona.listlayers('{}/{}'.format(folder, file))[0].endswith('slr_{}ft'.format(slr_feet)):
            return file

In [21]:
def create_slr_df(slr, slr_feet):
    '''Input: slr file stem and level of slr_feet rise
        Output: a dataframe made from the geospatial layer corresponding to the slr_feet rise parameter'''
    try:
        folder = 'data/SLR_ShapeFiles/{}_slr_final_dist.gdb'.format(slr)
        slr_file = find_slr_layer(slr, folder, slr_feet)
        slr_df = gpd.read_file('{}/{}'.format(folder, slr_file))
    except:
        try:
            #A number of files are not separate .gdbtable files, but .gpkg files where each slr scenario is a layer
            slr_df = gpd.read_file('data/SLR_ShapeFiles/{}_slr_final_dist.gpkg'.format(slr), 
            layer='{}_slr_{}_0ft'.format(slr, slr_feet))
            
        #Handling exceptions for 2 files that do not adhere to same file formatting
        except:
            try:
                slr_df = gpd.read_file('data/SLR_ShapeFiles/{}_slr_final_dist.gpkg'.format(slr), 
                layer='FL_East1_slr_{}_0ft'.format(slr_feet))
            except:
                folder = 'data/SLR_ShapeFiles/LA_LakePontchartrain_slr_final_dist_polys.gdb'
                slr_file = find_slr_layer(slr, folder, slr_feet)
                slr_df = gpd.read_file('{}/{}'.format(folder, slr_file))
                
    return slr_df

In [22]:
def create_intersecting_df(slr_df, census_df, slr, slr_feet):
    '''Input: SLR geospatial dataframe and Census tract geospatial dataframe
    Output: New temp_df containing all the geospatial locations that intersect - 
    that is, the census tracts that are affected by the specific SLR scenario.'''
    
    #Saving index so can reference which tract 'shapes' intersect
    census_df['savedindex'] = census_df.index 
    intersecting = slr_df.sjoin(census_df, how='inner')['savedindex']
        
    #Creating new DF
    temp_df = census_df[census_df.savedindex.isin(intersecting)][['STATEFP','COUNTYFP','TRACTCE']]
    temp_df['SLRs'] = slr
    temp_df['census_tract']= temp_df.apply(lambda x: x['STATEFP']+x['COUNTYFP']+x['TRACTCE'], axis=1)
    temp_df['SLR_ft'] = slr_feet
    
    return temp_df

In [23]:
def create_slr_csv(slr_feet, SLRs, fips_dict):
    '''Input: slr_feet - a specific sea level rise scenario, 
        SLRs - list of file stems from NOAA website for county areas to use
        fips_dict - dictionary of state-FIPS numbers for states to use
       Output: saves a .csv file with all the census tracts that are effected by slr scenario'''
    SLR_Census_df = pd.read_csv('data/SLR.csv')
    
    for stem in SLRs:
        slr = stem.split('/')[1]
        state = slr[:2]
        #Pass if already in CSV - this way can re-run if process fails or times out
        if slr in SLR_Census_df[SLR_Census_df.SLR_ft == int(slr_feet)].SLRs.unique():
            pass
        #Otherwise create geopandas df for specified slr scenario
        else:
            slr_df = create_slr_df(slr,slr_feet)
            census_df = gpd.read_file("data/Census_ShapeFiles/tl_rd22_{}_tract.shp".format(fips_dict[state]))
            temp_df = create_intersecting_df(slr_df, census_df, slr, slr_feet)
        
            #Updating SLR_Census df and saving to csv
            SLR_Census_df = pd.concat([SLR_Census_df, temp_df], axis=0)
            SLR_Census_df.to_csv('data/SLR.csv', index=False)
            
            #Uncomment for print statements to track file download progress.
            print(slr, 'added')

In [25]:
create_slr_csv(slr_feet, SLRs, fips_dict)